<a href="https://colab.research.google.com/github/guscldns/dacon-/blob/main/%EC%B5%9C%EC%A2%85_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 팀소개: 네모시

<<이런 저런 내용들>>

이름 끝자가 모두 모음으로 시작! (연 원 우 운)

네모시 = 네 명의 이름 마지막 글자가 모음으로 시작

# [요약] 주요 포인트

<<이런 저런 내용들>>

# 준비단계

## Google Drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## train, test 파일 다운로드

In [ ]:
!gdown https://drive.google.com/uc?id=1F6Otr01l-sUUR_1AdTM925tAGDE_L--t
!gdown https://drive.google.com/uc?id=13lNIGs3wsD0mZF4OauV4GhZQHniNjK6d

Downloading...
From: https://drive.google.com/uc?id=1F6Otr01l-sUUR_1AdTM925tAGDE_L--t
To: /content/train.csv
100% 3.47M/3.47M [00:00<00:00, 204MB/s]
Downloading...
From: https://drive.google.com/uc?id=13lNIGs3wsD0mZF4OauV4GhZQHniNjK6d
To: /content/test.csv
100% 1.28M/1.28M [00:00<00:00, 170MB/s]


## 패키지 설치

In [ ]:
!pip install category_encoders

## 패키지 불러오기

In [ ]:
# Core
import os, math
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import missingno as msno

# scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold, cross_validate, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
from category_encoders.ordinal import OrdinalEncoder

# 분류 모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb

# utils
import warnings
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')
tqdm.pandas()

## csv 불러오기

In [ ]:
df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

# EDA(Exploratory Data Analysis, 탐색적 데이터 분석)

### **EDA란?**
> EDA 혹은 탐색적 데이터 분석이라고도 불리는 이 절차는, 데이터를 살펴보고 뜯어보는 과정에서 반복되는 패턴이나 관계를 도출해내고 인사이트를 얻기 위해 필요합니다.
    <br>
    <br>
    이미 잘 아는 분야, 데이터라면 "이렇게 하면 이런 결과가 나올 것이다"라는 가설을 가지고 바로 실험해볼 수도 있지만,
    그렇지 않을 때는 반드시 거쳐야 하는 과정입니다. 물론 잘 아는 분야여도 EDA를 통해서 새로운 정보를 얻을 수도 있구요!
    <br>
    <br>
    코드를 작성해야 하는 일이 늘어나 부담이 될 수는 있지만, 섣불리 건너뛰었다가는 데이터에 대한 이해 부족으로 다시 EDA를 하러 돌아오는 불상사가 발생할 수 있으니,
    조금만 힘내서 EDA를 해보자구요.
    
<br>
<br>

---

##### >> EDA가 궁금하신 분들을 위해, 데이콘에 소개된 양질의 게시글들을 추천해드릴게요!

| URL                                                                                                                                  | 작성자    | 추천사유                                                                                                                                                                                       |
|--------------------------------------------------------------------------------------------------------------------------------------|-----------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| [[EDA] 데이터 분석 입문자를 위한 데이터 살펴보기](https://dacon.io/competitions/official/236147/codeshare/8682)                      | DACONIO   | 데이콘에서 제공하는 EDA baseline 코드입니다.  <br><br> 데이터와 주제는 다르지만, EDA의 필수요소라고 할 수 있는 내용들이  <br>들어가있기 때문에 추천드려요!                                     |
| [[데이썬☀️_3편] 🔍EDA (탐색적 데이터 분석) 사용 설명서 (1) - EDA & 통계치 분석](https://dacon.io/codeshare/4899)                       | 므인      | 기본적인 EDA의 종류와 유형부터 이상치 처리, 결측치와 같은 통계치 분석에 대한 내용도 일목요연하게 정리되어 있습니다. <br><br> 데이터 분석 입문자 분들께는 좋은 참고자료가 될 것이기에 추천드립니다. |
| [ 데이콘의 경진대회 수상작들 중에 PDF로 만들어진 내용을 슬라이드 쉐어에 모아 봤습니다. ](https://dacon.io/forum/409145?dtype=recent) | DATA MONK | 해당 게시글에 시각화 우수 사례들이 많이 소개되어 있습니다.  <br><br> 실제 프로젝트, 대회에서는 어떻게 데이터를 시각화하는지 궁금하다면 꼭 확인해보세요.                                            |

In [ ]:
# 승연님 자료가 들어갈 예정

# Feature Selection / Feature Engineering


 ### **Feature Selection이란?**
> EDA 혹은 모델 성능을 확인하는 과정에서 필요한 feature를 선별하는 절차를 뜻합니다.
    <br>
    <br>
    가령 모델 성능을 비교해보는 과정에서, 'A'라는 feature를 없앴을 때 성능이 더 좋아서 빼기로 결정했다면, 이도 feature selection을 수행한 것이지요.

<br>

### **Feature Engineering이란?**
> 그러면 Feature Engineering은 무엇일까요? 간혹 결측치, 중복값, 이상치, 데이터 편향 등의 이유로 feature selection만으로는 적절한 모델 성능을 기대하기 어려울 때가 있습니다.
    <br>
    <br>
    이럴 때 feature를 조작하는 것을 feature engineering이라고 합니다.  결측치를 특정 값으로 채워줄 수도 있고, 결측치가 있는 행/열을 없애줄 수도 있죠.
    <br>
    앞서 EDA에서 특정 feature끼리 결합해서 새로운 feature를 만드는 것도 모두 이에 해당됩니다.
    <br>
    주어진 feature만으로는 성능이 더이상 오르지 않는다면, 새로운 파생변수를 만드는 것도 고려해보세요!


## 불필요한 feature 삭제

In [ ]:
df = df.drop(['FLAG_MOBIL', 'index'], axis=1)
test_df = test_df.drop(['FLAG_MOBIL', 'index'], axis=1)

##### 설명

* 'FLAG_MOBIL' : 모든 데이터에서 1로 동일한 값을 가지기 때문에, 변수 간 관계 파악에 도움이 되지 않습니다.
* 'index' : 모든 데이터에서 서로 다른 값을 가지며, 특정 기준에 의한 값이 아닌 단순히 행 번호를  나타냅니다. 따라서 마찬가지로 변수 간 관계 파악에 도움이 되지 않습니다.

## 결측치 처리

In [ ]:
df['occyp_type'] = df['occyp_type'].fillna("Unknown")
test_df['occyp_type'] = test_df['occyp_type'].fillna("Unknown")

##### 설명

데이터의 수집 배경을 추측해보는 과정에서, 가입 시 고객으로부터 입력받은 개인정보라고 가정하고 접근했습니다.
<br>
이 때 직업을 미기입한 경우 결측치로 나타난다고 유추할 수 있습니다.
<br>
<br>
결측치가 데이터 손실이 아닌 의도된 것이라면, 이를 'Unknown'으로 일괄적으로 채워서, 하나의 카테고리로 인식하도록 합니다.

## 파생변수: ID

In [ ]:
add_columns = ['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type',
       'house_type', 'occyp_type', 'work_phone', 'email', 'DAYS_BIRTH', 'child_num', 'DAYS_EMPLOYED']

df['ID'] = df[add_columns].astype(str).apply(lambda x: '_'.join(x), axis=1)
test_df['ID'] = test_df[add_columns].astype(str).apply(lambda x: '_'.join(x), axis=1)

##### 설명

먼저 drop한 'index', 'FLAG_MOBIL'을 제외하면, 'income_total', 'phone', 'family_size', 'begin_month', 'credit' 외의 모든 변수를 문자열로 더해주었습니다.
<br>
- 이 때 각각의 데이터(row)는 기본적으로는 고유한 값을 가지게 되며, 겹치는 ID값은 동일인물로 식별되도록 하였습니다.
<br>
- 가입 시기('begin_month')에 따라 수입 수준('income_total'), 가족구성원 수('family_size') 등의 정보는 변동될 수 있다고 가정했습니다.
<br>
- credit은 예측해야 하는 y값이기 때문에 제외되었습니다.

## 'DAYS_EMPLOYED' 핸들링

### 이상치 처리

In [ ]:
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].apply(lambda x: 0 if x > 0 else x)
test_df['DAYS_EMPLOYED'] = test_df['DAYS_EMPLOYED'].apply(lambda x: 0 if x > 0 else x)

##### 설명

근무일수('DAYS_EMPLOYED')는 시작일을 기준으로 경과된 일자만큼의 음수값을 지닙니다. 하지만 양수인 데이터가 있는데, 이들은 이상치로 간주하여, 모두 0으로 처리합니다.


## feature 핸들링: 음수 -> 절대값

In [ ]:
features = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']
for f in features:
    df[f] = df[f].abs()
    test_df[f] = test_df[f].abs()

##### 설명

전반적으로 절대값을 취했을 때 성능이 더 좋았기에, 이를 적용합니다.

## 파생변수: no_work

In [ ]:
df["No_work"] = df["DAYS_BIRTH"] - df['DAYS_EMPLOYED']
test_df["No_work"] = test_df["DAYS_BIRTH"] - test_df['DAYS_EMPLOYED']

##### 설명
출생 이후부터 취업 전까지를 일하지 않은 날로 간주하여 'DAYS_BIRTH'에서 'DAYS_EMPLOYED'를 빼주었습니다.

## 파생변수: 연, 월, 주

In [ ]:
def get_DAY_MONTH_WEEK(df, feature):
    df[f'{feature}(year)'] = df[feature] // 365 # 연 정보
    df[f'{feature}(month)'] = (df[feature] % 365) // 30 # 월 정보
    df[f'{feature}(week)'] = ((df[feature] % 365) % 30) // 7 # 일 정보

    return df

##### 설명
'DAYS_BIRTH', 'DAYS_EMPLOYED', 'No_work'의 경우 각각의 값을 일 단위 날짜로 표기했으며, 모델 성능 개선을 위해 각각 연, 월, 주 정보를 담은 파생변수를 생성합니다.

In [ ]:
# train data
df = get_DAY_MONTH_WEEK(df, 'DAYS_BIRTH')
df = get_DAY_MONTH_WEEK(df, 'DAYS_EMPLOYED')
df = get_DAY_MONTH_WEEK(df, 'No_work')

# test data도 똑같이 적용해줍니다.
test_df = get_DAY_MONTH_WEEK(test_df, 'DAYS_BIRTH')
test_df = get_DAY_MONTH_WEEK(test_df, 'DAYS_EMPLOYED')
test_df = get_DAY_MONTH_WEEK(test_df, 'No_work')

In [ ]:
# 변수 생성이 잘 되었는지 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   gender                26457 non-null  object 
 1   car                   26457 non-null  object 
 2   reality               26457 non-null  object 
 3   child_num             26457 non-null  int64  
 4   income_total          26457 non-null  float64
 5   income_type           26457 non-null  object 
 6   edu_type              26457 non-null  object 
 7   family_type           26457 non-null  object 
 8   house_type            26457 non-null  object 
 9   DAYS_BIRTH            26457 non-null  int64  
 10  DAYS_EMPLOYED         26457 non-null  int64  
 11  work_phone            26457 non-null  int64  
 12  phone                 26457 non-null  int64  
 13  email                 26457 non-null  int64  
 14  occyp_type            26457 non-null  object 
 15  family_size        

## 파생변수: 'income_year'

In [ ]:
df['income_year'] = df['income_total'] / 365
test_df['income_year'] = test_df['income_total'] / 365
# df['income_year'] = df['income_total'] / df['DAYS_EMPLOYED(year)']
# test_df['income_year'] = test_df['income_total'] / test_df['DAYS_EMPLOYED(year)']

##### 설명

<<채울 예정>>

## 파생변수: 'income'

In [ ]:
df['income'] = df['income_total'] / ((df['DAYS_BIRTH'] + df['DAYS_EMPLOYED']))
test_df['income'] = test_df['income_total'] / ((test_df['DAYS_BIRTH'] + test_df['DAYS_EMPLOYED']))

##### 가정
나이가 많으면서 근무일수도 높다면, 소득수준이 더 높을 것이라고 가정하여 접근했습니다.
<br>
<br>
##### 설명
총소득('income_total')을 살아온 일수('DAYS_BIRTH') + 근무일수('DAYS_EMPLOYED')로 나눠주었습니다.


## 파생변수: 'income_family'

In [ ]:
df['income_famliy'] = df['income_total'] / df['family_size']
test_df['income_famliy'] = test_df['income_total'] / test_df['family_size']

##### 가정
가족구성원이 많다면 실질적인 체감소득이 적을 것이라고 가정했습니다.
<br>
<br>

##### 설명
소득은 동일하되 가족구성원 수가 더 많은 경우, 체감소득은 낮아질 것이기 때문에 총소득('income_total')을 가족구성원 수('family_size')로 나눠주었습니다.

## 파생변수: car + email

In [ ]:
df['car_email'] = df.loc[:, 'car':'email'].sum(axis = 1)
test_df['car_email'] = test_df.loc[:, 'car':'email'].sum(axis = 1)

df = df.drop(['car', 'email'], axis=1)
test_df = test_df.drop(['car', 'email'], axis=1)

##### 설명

'car', 'email' 자체로는 모델 성능에 큰 영향을 끼치지 못했으나, 둘을 더한 파생변수를 사용했을 때는 성능 개선 효과가 있었습니다.

## 불필요 변수 삭제 << 나중에 child_num 살려서 써보기 >>

In [ ]:
df = df.drop(['DAYS_EMPLOYED','DAYS_BIRTH','child_num'], axis=1)
test_df = test_df.drop(['DAYS_EMPLOYED','DAYS_BIRTH','child_num'], axis=1)

# 전처리

<< 설명 보충 필요 >>

## 타입캐스팅

In [ ]:
df['credit'] = df['credit'].astype(int)

## train validation split

In [ ]:
X_data = df.drop('credit', axis=1)
y_data = df['credit']

X_train, X_val, y_train, y_val = train_test_split(X_data, y_data,
                                                  test_size=0.2,
                                                  random_state=777,
                                                  stratify=y_data
                                                  )

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((21165, 27), (5292, 27), (21165,), (5292,))

## OrdinalEncoder

In [ ]:
# object열 확인하기
object_columns = X_train.dtypes[X_train.dtypes == "object"].index.tolist()
print(object_columns)

['gender', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'ID']


In [ ]:
# object열 인코딩하기
orde = OrdinalEncoder(object_columns)
X_train[object_columns] = orde.fit_transform(X_train[object_columns])
X_val[object_columns] = orde.transform(X_val[object_columns])
test_df[object_columns] = orde.transform(test_df[object_columns])

In [ ]:
orde.feature_names_in_

array(['gender', 'reality', 'income_type', 'edu_type', 'family_type',
       'house_type', 'occyp_type', 'ID'], dtype=object)

# 모델링: LGBM

In [ ]:
lgbm = LGBMClassifier(n_estimators=300, boosting_type='gbdt', learning_rate=0.1,
                    objective='multiclass', random_state=777, n_jobs=4,)

lgbm.fit(X_train, y_train)

lgbm.score(X_val, y_val)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002425 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1886
[LightGBM] [Info] Number of data points in the train set: 21165, number of used features: 27
[LightGBM] [Info] Start training from score -2.105335
[LightGBM] [Info] Start training from score -1.440314
[LightGBM] [Info] Start training from score -0.444193


0.7178760393046107

In [ ]:
import time, datetime
START = datetime.datetime.now()
print(f"start: {START.strftime('%H:%M:%S')}")

params_grid = {
    'n_estimators' : [200, 250, 300, 350, 400, 450, 500],
    'boosting_type' : ['gdbt', 'dart', 'rf'],
    'learning_rate' : [0.0, 0.001, 0.005, 0.1, 0.3, 0.5, 0.75, 1.0],
    'max_depth' : list(range(1, 10)) + [-1],
}

grid_cv = GridSearchCV(
    estimator = LGBMClassifier(objective='multiclass', random_state=777),
    param_grid = params_grid,
    scoring = ['accuracy', 'f1'],
    refit = 'accuracy',
    n_jobs = -1,
    cv = 10,
    verbose = 3
)

result = grid_cv.fit(X_train, y_train)


END = datetime.datetime.now()
print(f"end: {END.strftime('%H:%M:%S')}")